# 가사 검색

[2020 카카오 공채 '가사 검색' 문제 링크](https://programmers.co.kr/learn/courses/30/lessons/60060)

### 문제  
친구들로부터 천재 프로그래머로 불리는 프로도는 음악을 하는 친구로부터 자신이 좋아하는 노래 가사에 사용된 단어들 중에 특정 키워드가 몇 개 포함되어 있는지 궁금하니 프로그램으로 개발해 달라는 제안을 받았습니다.  
그 제안 사항 중, 키워드는 와일드카드 문자중 하나인 '?'가 포함된 패턴 형태의 문자열을 뜻합니다. 와일드카드 문자인 '?'는 글자 하나를 의미하며, 어떤 문자에도 매치된다고 가정합니다. 예를 들어 "fro??"는 "frodo", "front", "frost" 등에 매치되지만 "frame", "frozen"에는 매치되지 않습니다.  
  
가사에 사용된 모든 단어들이 담긴 배열 words와 찾고자 하는 키워드가 담긴 배열 queries가 주어질 때, 각 키워드 별로 매치된 단어가 몇 개인지 순서대로 배열에 담아 반환하도록 solution 함수를 완성해 주세요.  
  
### 가사 단어 제한사항  
- words의 길이(가사 단어의 개수)는 2 이상 100,000 이하입니다.  
- 각 가사 단어의 길이는 1 이상 10,000 이하로 빈 문자열인 경우는 없습니다.  
- 전체 가사 단어 길이의 합은 2 이상 1,000,000 이하입니다.  
- 가사에 동일 단어가 여러 번 나올 경우 중복을 제거하고 words에는 하나로만 제공됩니다.  
- 각 가사 단어는 오직 알파벳 소문자로만 구성되어 있으며, 특수문자나 숫자는 포함하지 않는 것으로 가정합니다.  

### 검색 키워드 제한사항
- queries의 길이(검색 키워드 개수)는 2 이상 100,000 이하입니다.
- 각 검색 키워드의 길이는 1 이상 10,000 이하로 빈 문자열인 경우는 없습니다.
- 전체 검색 키워드 길이의 합은 2 이상 1,000,000 이하입니다.
- 검색 키워드는 중복될 수도 있습니다.
- 각 검색 키워드는 오직 알파벳 소문자와 와일드카드 문자인 '?' 로만 구성되어 있으며, 특수문자나 숫자는 포함하지 않는 것으로 가정합니다.
- 검색 키워드는 와일드카드 문자인 '?'가 하나 이상 포함돼 있으며, '?'는 각 검색 키워드의 접두사 아니면 접미사 중 하나로만 주어집니다.
- 예를 들어 "??odo", "fro??", "?????"는 가능한 키워드입니다.
- 반면에 "frodo"('?'가 없음), "fr?do"('?'가 중간에 있음), "?ro??"('?'가 양쪽에 있음)는 불가능한 키워드입니다.  

In [1]:
class Node():# 트라이를 구성할 노드
    def __init__(self, key, is_word=False):
        self.key = key # 노드가 나타내는 문자
        self.is_word = is_word # 단어의 끝 -> True일 경우 단어가 구성됨
        self.num = 0 # 해당 문자를 지나치는 단어들의 개수
        self.children = {} # 해당 문자 다음에 올수 있는 문자들
        
class Trie(): # 트라이구조
    def __init__(self):
        self.heads = {} # 단어의 길이별 단어를 다르게 트라이를 구성할 것임
    
    def insert(self, string):
        len_s = len(string) # 문자열 길이
        curr_node = None
        if self.heads.get(len_s):
            curr_node = self.heads[len_s]
        else:
            curr_node = Node(None)
            self.heads[len_s] = curr_node
        # 해당하는 길이의 단어 트라이 구성
        curr_node.num += 1
        
        for s in string: # 문자열에 대해서
            next_node = None
            if curr_node.children.get(s):
                next_node = curr_node.children[s]
            else:
                next_node = Node(s)
            # 다음 문자에 대한 노드를 갖고오거나 없다면 생성
                curr_node.children[s] = next_node
                # 현재 노드 다음에 올 수 있는 문자로 s를 설정
            next_node.num += 1 # 해당 노드를 거치는 문자 개수 + 1
            curr_node = next_node # 다시 다음 노드에 대해서 똑같이 진행
        curr_node.is_word = True # 단어가 끝나는 노드에는 표시
      
    
    def start_with(self, prefix, length):
        if self.heads.get(length): # 해당 길이에 맞는 트라이 선택
            curr_node = self.heads[length]
        else: # 해당 길이의 트라이가 없다면 0
            return 0
        for p in prefix: # prefix에 대해서
            if curr_node.children.get(p):
                curr_node = curr_node.children[p] # 트라이 탐색 진행
            else:
                return 0 # 탐색이 안된다면 0
        return curr_node.num # 탐색 끝난 후 해당 노드를 거친 단어의 개수 반환
                

def solution(words, queries):
    answer = [0 for _ in range(len(queries))]
    front = Trie()
    back = Trie()
    for i in words: # 모든 단어에 대해
        front.insert(i) # 트라이에 넣는다.
        back.insert(i[::-1]) # 단어의 역순으로 트라이에 넣는다.
    for j in range(len(queries)):
        len_q = len(queries[j])
        if queries[j][0] == '?': # 만약 postfix라면
            answer[j] += back.start_with(queries[j][::-1].split('?')[0], len_q)
            # 역순으로 구성한 트라이에 대해 트라이 탐색 진행
        else: # prefix라면
            answer[j] += front.start_with(queries[j].split('?')[0], len_q)
            # 원래 순서대로 구성한 트라이에 대해 트라이 탐색 진행
    return answer

In [2]:
words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "????o", "fr???", "fro???", "pro?"]

In [3]:
solution(words, queries)

[3, 2, 4, 1, 0]